In [22]:
import geopandas as gpd
import momepy as mm  # requires main
import networkx as nx

In [2]:
auckland = gpd.read_parquet("../data/869/roads_osm.parquet")

Simplification using `roundabout_simplification` method to tackle roundabouts.

In [3]:
simple = mm.roundabout_simplification(auckland)

/Users/martin/Git/momepy/momepy/preprocessing.py:762: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.593543449979414' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rab_adj.loc[g.Index, "hdist"] = hdist
/Users/martin/Git/momepy/momepy/preprocessing.py:858: FutureWarning: The `query_bulk()` method is deprecated and will be removed in GeoPandas 1.0. You can use the `query()` method instead.
  edges_idx, rabs_idx = rab_multipolygons.sindex.query_bulk(


The function has quite a few parameters and fairly smart logic of determining the _main road_ using COINS algorithm.

In [6]:
mm.roundabout_simplification?

Signature:
mm.roundabout_simplification(
    edges,
    polys=None,
    area_col='area',
    circom_threshold=0.7,
    area_threshold=0.85,
    include_adjacent=True,
    diameter_factor=1.5,
    center_type='centroid',
    angle_threshold=0,
)
Docstring:
Selects the roundabouts from ``polys`` to create a center point to merge all
incoming edges. If None is passed, the function will perform shapely polygonization.

All ``edges`` attributes are preserved and roundabouts are deleted.
Note that some attributes, like length, may no longer reflect the reality of newly
constructed geometry.

If ``include_adjacent`` is True, adjacent polygons to the actual roundabout are
also selected for simplification if two conditions are met:
    - the area of adjacent polygons is less than the actual roundabout
    - adjacent polygons do not extend beyond a factor of the diameter of the actual
    roundabout.
    This uses hausdorff_distance algorithm.

Parameters
----------
edges : GeoDataFrame
    GeoD

The other option is to use `consolidate_intersections` which works more based on graph properties.

In [17]:
graph = mm.gdf_to_nx(auckland, length="length", integer_labels=True)

We need to add additional edge attributes _from_ and _to_ at the moment. Seems redundant, will need to refactor.

In [23]:
nx.set_edge_attributes(
    graph, {edge: {"from": edge[0], "to": edge[1]} for edge in graph.edges}
)

In [24]:
simple_graph = mm.consolidate_intersections(graph)

It does not have a ton of relevant options but shall work better in preserving planarity I think.